In [2]:
GROQ_API_KEY = "ABCD"

In [3]:
from groq import Groq
import os
import fitz  # PyMuPDF

# Initialize Groq API Client
client = Groq(api_key=os.getenv('GROQ_API_KEY'))

In [5]:
def extract_text_from_pdf(pdf_path):
    """Extract text from a given PDF file."""
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

In [6]:
def split_text(text, max_tokens=2000):
    """Splits text into smaller chunks to fit within token limits."""
    words = text.split()
    chunks = []
    chunk = []

    for word in words:
        chunk.append(word)
        if len(chunk) >= max_tokens:  # Approximate token count
            chunks.append(" ".join(chunk))
            chunk = []

    if chunk:
        chunks.append(" ".join(chunk))

    return chunks

In [8]:
def summarize_text(text):
    """Summarize the extracted text before classification to reduce token size."""
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an AI that summarizes long text efficiently."},
            {"role": "user",
             "content": f"Summarize this text concisely while keeping key bias-related information:\n\n{text}"}
        ],
        model="llama-3.3-70b-versatile",  # Use a model optimized for summarization
        temperature=0.5,
        max_completion_tokens=512,
        top_p=1,
        stop=None,
        stream=False,
    )

    return chat_completion.choices[0].message.content

In [8]:
# def classify_bias(text):
#     """Classify bias using Groq API."""
#     chat_completion = client.chat.completions.create(
#         messages=[
#             {"role": "system", "content": "You are an AI assistant trained to classify biases in text."},
#             {"role": "user", "content": f"""
#                 Classify the biases judgment under one of the following criteria:
#                 A. Low risk of bias
#                 B. Unclear risk of bias
#                 C. High risk of bias
#
#                 In addition, provide a 'support for judgement' for any class (A, B, or C) chosen.
#
#                 Focus specifically on:
#                 1. Random sequence generation (selection bias)
#                 2. Allocation concealment (selection bias)
#                 3. Blinding of participants and personnel (performance bias)
#                 4. Blinding of outcome assessment (detection bias)
#                 5. Incomplete outcome data (attrition bias)
#                 6. Selective reporting (reporting bias)
#                 7. Other bias
#
#                 Text:
#                 {text}
#
#                 Respond in JSON format for each of the above (1,2,3,4,5,6,7):
#                 {{
#                     "classification": "<Low/Unclear/High> risk of bias",
#                     "support": "<justification>"
#                 }}
#             """}
#         ],
#         model="deepseek-r1-distill-qwen-32b",
#         temperature=0.5,
#         max_completion_tokens=512,  # Reduce to prevent token limit errors
#         top_p=1,
#         stop=None,
#         stream=False,
#     )
#
#     return chat_completion.choices[0].message.content

In [9]:
# %%capture cap1
# # Main Execution
# if __name__ == "__main__":
#     pdf_path = "01_NEJMoa2034577.pdf"  # Change to your actual PDF path
#     text = extract_text_from_pdf(pdf_path)
#
#     chunks = split_text(text)  # Split into smaller chunks
#     summarized_chunks = [summarize_text(chunk) for chunk in chunks]  # Summarize each chunk
#     results = []
#
#     for idx, chunk in enumerate(summarized_chunks):
#         print(f"Processing chunk {idx + 1}/{len(summarized_chunks)}...")
#         result = classify_bias(chunk)
#         results.append(result)
#
#     print("\nBias Classification Results:")
#     for res in results:
#         print(res)
#
# # Save the captured output to a text file
# with open('output1-NEJMoa2034577.txt', 'w') as file:
#     file.write(cap1.stdout)

Fix Biases

In [34]:
# %%capture cap1
#
# import json
# import re
# from groq import Groq
# import fitz  # PyMuPDF
#
# # Initialize Groq API Client
# # client = Groq(api_key="your_groq_api_key")
#
# BIAS_TYPES = [
#     "Random sequence generation (selection bias)",
#     "Allocation concealment (selection bias)",
#     "Blinding of participants and personnel (performance bias)",
#     "Blinding of outcome assessment (detection bias)",
#     "Incomplete outcome data (attrition bias)",
#     "Selective reporting (reporting bias)",
#     "Other bias"
# ]
#
# def extract_text_from_pdf(pdf_path):
#     """Extract text from a given PDF file."""
#     doc = fitz.open(pdf_path)
#     text = "\n".join([page.get_text("text") for page in doc])
#     return text
#
# def split_text(text, max_tokens=2000):
#     """Splits text into smaller chunks to fit within token limits."""
#     words = text.split()
#     chunks = []
#     chunk = []
#
#     for word in words:
#         chunk.append(word)
#         if len(chunk) >= max_tokens:
#             chunks.append(" ".join(chunk))
#             chunk = []
#
#     if chunk:
#         chunks.append(" ".join(chunk))
#
#     return chunks
#
# def extract_json_from_text(response_text):
#     """Extract JSON content from the response using regex."""
#     match = re.search(r"\{.*\}", response_text, re.DOTALL)
#     if match:
#         return match.group(0)  # Return only the JSON part
#     return None  # No valid JSON found
#
# def classify_bias(text):
#     """Classify bias for all 7 categories using Groq API."""
#
#     bias_results = {}
#
#     for bias_type in BIAS_TYPES:
#         chat_completion = client.chat.completions.create(
#             messages=[
#                 {"role": "system", "content": "You are an AI assistant trained to classify biases in text."},
#                 {"role": "user", "content": f"""
#                     Analyze the following text and classify the bias under one of the following categories:
#                     A. Low risk of bias
#                     B. Unclear risk of bias
#                     C. High risk of bias
#
#                     In addition, provide a 'support for judgement' explaining your classification.
#
#                     Focus specifically on: {bias_type}
#
#                     Text:
#                     {text}
#
#                     Respond in JSON format:
#                     {{
#                         "classification": "<Low/Unclear/High> risk of bias",
#                         "support": "<justification>"
#                     }}
#                 """}
#             ],
#             model="deepseek-r1-distill-qwen-32b",
#             temperature=0.5,
#             max_completion_tokens=512,
#             top_p=1,
#             stop=None,
#             stream=False,
#         )
#
#         response_text = chat_completion.choices[0].message.content
#
#         try:
#             json_text = extract_json_from_text(response_text)  # Extract only JSON part
#             if json_text:
#                 response_json = json.loads(json_text)
#                 bias_results[bias_type] = response_json
#             else:
#                 bias_results[bias_type] = {"error": "Invalid JSON format returned"}
#
#         except json.JSONDecodeError:
#             bias_results[bias_type] = {"error": "Failed to parse JSON response"}
#
#     return bias_results
#
#
# # Main Execution
# if __name__ == "__main__":
#     pdf_path = "01_NEJMoa2034577.pdf"  # Change to your actual PDF path
#     text = extract_text_from_pdf(pdf_path)
#
#     chunks = split_text(text)  # Split into smaller chunks
#     all_results = {}
#
#     for idx, chunk in enumerate(chunks):
#         print(f"Processing chunk {idx + 1}/{len(chunks)}...")
#         bias_results = classify_bias(chunk)
#
#         # Merge results from multiple chunks
#         for key, value in bias_results.items():
#             if key not in all_results:
#                 all_results[key] = []
#             all_results[key].append(value)
#
#     print("\nFinal Bias Classification Results:")
#     print(json.dumps(all_results, indent=4))
#
# with open('01_NEJMoa2034577.txt', 'w') as file:
#     file.write(cap1.stdout)

Save to txt

In [9]:
import json
import re
from groq import Groq
import fitz  # PyMuPDF

# Initialize Groq API Client
# client = Groq(api_key="xxx")

BIAS_TYPES = [
    "Random sequence generation (selection bias)",
    "Allocation concealment (selection bias)",
    "Blinding of participants and personnel (performance bias)",
    "Blinding of outcome assessment (detection bias)",
    "Incomplete outcome data (attrition bias)",
    "Selective reporting (reporting bias)",
    "Other bias"
]

def extract_text_from_pdf(pdf_path):
    """Extract text from a given PDF file."""
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

def split_text(text, max_tokens=2000):
    """Splits text into smaller chunks to fit within token limits."""
    words = text.split()
    chunks = []
    chunk = []

    for word in words:
        chunk.append(word)
        if len(chunk) >= max_tokens:
            chunks.append(" ".join(chunk))
            chunk = []

    if chunk:
        chunks.append(" ".join(chunk))

    return chunks

def extract_json_from_text(response_text):
    """Extract JSON content from the response using regex."""
    match = re.search(r"\{.*\}", response_text, re.DOTALL)
    if match:
        return match.group(0)  # Return only the JSON part
    return None  # No valid JSON found

def classify_bias(text):
    """Classify bias for all 7 categories using Groq API."""

    bias_results = {}

    for bias_type in BIAS_TYPES:
        chat_completion = client.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are an AI assistant trained to classify biases in text."},
                {"role": "user", "content": f"""
                    Analyze the following text and classify the bias under one of the following categories:
                    A. Low risk of bias
                    B. Unclear risk of bias
                    C. High risk of bias

                    In addition, provide a 'support for judgement' explaining your classification.

                    Focus specifically on: {bias_type}

                    Text:
                    {text}

                    Respond in JSON format:
                    {{
                        "classification": "<Low/Unclear/High> risk of bias",
                        "support": "<justification>"
                    }}
                """}
            ],
            model="deepseek-r1-distill-qwen-32b",
            temperature=0.5,
            max_completion_tokens=512,
            top_p=1,
            stop=None,
            stream=False,
        )

        response_text = chat_completion.choices[0].message.content

        try:
            json_text = extract_json_from_text(response_text)  # Extract only JSON part
            if json_text:
                response_json = json.loads(json_text)
                bias_results[bias_type] = response_json
            else:
                bias_results[bias_type] = {"error": "Invalid JSON format returned"}

        except json.JSONDecodeError:
            bias_results[bias_type] = {"error": "Failed to parse JSON response"}

    return bias_results


def save_results_to_txt(results, output_file="./output/01_PIIS2666776221002465_bias_classification_results.txt"):
    """Save classification results to a text file."""
    with open(output_file, "w", encoding="utf-8") as file:
        file.write("Bias Classification Results\n")
        file.write("=" * 50 + "\n\n")

        for bias_type, assessments in results.items():
            file.write(f"{bias_type}:\n")
            for assessment in assessments:
                file.write(f"  Classification: {assessment.get('classification', 'N/A')}\n")
                file.write(f"  Support: {assessment.get('support', 'N/A')}\n")
                file.write("\n")

        print(f"\nResults saved to {output_file}")

In [10]:
# Main Execution
if __name__ == "__main__":
    pdf_path = "./articles/01_PIIS2666776221002465.pdf"  # PDF path
    text = extract_text_from_pdf(pdf_path)

    chunks = split_text(text)  # Split into smaller chunks
    all_results = {}

    for idx, chunk in enumerate(chunks):
        print(f"Processing chunk {idx + 1}/{len(chunks)}...")
        bias_results = classify_bias(chunk)

        # Merge results from multiple chunks
        for key, value in bias_results.items():
            if key not in all_results:
                all_results[key] = []
            all_results[key].append(value)

    # Save to text file
    save_results_to_txt(all_results)

Processing chunk 1/4...
Processing chunk 2/4...
Processing chunk 3/4...
Processing chunk 4/4...

Results saved to ./output/01_PIIS2666776221002465_bias_classification_results.txt
